# Load libraries

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns


# Load data and analize size and type of information in each column

In [7]:
# load data
bcn_df = pd.read_csv('../data/raw/barcelona_listings_kagel.csv')
#bcn_df.columns
#bcn_df.head
#print(bcn_df) # 19k rows  106 cols
print ('Understanding the data set')
print('The number of rows aand columns in the original data set is: ',bcn_df.shape)
print('The list of columns in the original data set is: ',bcn_df.columns)
#bcn_df.columns
print('The list of columns and their value types are: ', bcn_df.info())
#bcn_df.head()

"""
ALL COLUMN NAMES ARE ALREADY LOWER CASE AND HAVE NO WHITE SPACES
COLUMNS THAT CAN BE DIRECTLY REMOVED AS NO RELEVANT FOR THE ANALYSIS
COLUMN SQUARE FEET HARS ONLY 500 NON NULL VALUES SO IT NEEDS TO BE REMOVED
For example 'neighbourhood' and 'neighbourhood_cleansed' have the same information.
No need for country.
For 'square_feet' there are values for only 500 rows.
"""

columns_to_delete = ['picture_url','listing_url','host_url','host_name', 'host_has_profile_pic', 'host_picture_url','space','description','summary','access','neighborhood_overview','summary','country', 'host_has_profile_pic', 'square_feet','neighbourhood','street' ] 
bcn_df.drop(columns=columns_to_delete, inplace=True)

print('The number of rows and columns after deleting columns: ',bcn_df.shape)
bcn_df.nunique()
bcn_df.isna().sum()

"""
    RENAME SOME OF THE KEY COLUMNS
"""
bcn_df.rename(columns={'host_since': 'listed_date'}, inplace = True)
bcn_df.rename(columns={'neighbourhood_cleansed': 'neighbourhood'}, inplace = True)
bcn_df.rename(columns={'neighbourhood_group_cleansed': 'area'}, inplace = True)
print(bcn_df.info())

df1 = bcn_df.copy()

/var/folders/yl/d5x0r6t10l52bms12sy70rzc0000gn/T/ipykernel_69836/2671617568.py:2: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  bcn_df = pd.read_csv('../data/raw/barcelona_listings_kagel.csv')


Understanding the data set
The number of rows aand columns in the original data set is:  (19833, 62)
The list of columns in the original data set is:  Index(['id', 'listing_url', 'name', 'summary', 'space', 'description',
       'neighborhood_overview', 'access', 'picture_url', 'host_id', 'host_url',
       'host_name', 'host_since', 'host_response_time', 'host_response_rate',
       'host_is_superhost', 'host_picture_url', 'host_neighbourhood',
       'host_listings_count', 'host_verifications', 'host_has_profile_pic',
       'host_identity_verified', 'street', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'city',
       'zipcode', 'country', 'latitude', 'longitude', 'is_location_exact',
       'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms',
       'beds', 'amenities', 'square_feet', 'price', 'cleaning_fee',
       'minimum_nights', 'maximum_nights', 'has_availability',
       'availability_30', 'availability_60', 'availabilit

# Column 'city': cleaning wrong values and filling nulls.

In [8]:
"""
    CHECK FOR UNIQUE VALUES IN COLUMN "CITY" - REMOVE WHITE SPACES AND ALL LOWER CASE.
"""
#print(bcn_df['city'].unique())
def city_lowercase(df):
    df['city'] = df['city'].str.lower().str.strip() 
    return df
df1 = city_lowercase(df1)

# Check if the replacement was successful
#print(df1['city'].unique())

"""
    EXCLUDE AREAS OUTSIDE THE 10 AREAS OF BARCELONA
"""
values_to_delete = ['sant adria de besos', 'st cugat del vallès', 'барселона',"l'hospitalet de llobregat" ,'hospitalet de llobregat','03-jan', 'sant adrià de besòs','巴塞罗那' ,'sant cugat del vallès' , '巴塞罗纳','sant adrià de besos' ,'hospitalet','badalona']
# Filter DF to exclude rows with the specified values
df1 = df1[~df1['city'].isin(values_to_delete)]
#print(df1['city'].unique())

"""
    REMAINING VALUES NEED TO BE ALL "barcelona" - REPLACE MISSPELLING
"""
new_value = 'barcelona'
df1['city'] = df1['city'].replace(['barcelonaneta','барселона','sagrada familia, barcelona' ,'barcelone','bcn' ,'barcelona, catalunya, es', 'barcelona el raval' ,'zona forum','barcelon' ,'83-93' ,'08028, barcelona' ,'gracia-barcelona', '.','sants-les corts' ,'eixample', 'les corts' ,'*','maragall- hospital de sant pau' ,'sants'], new_value)
#print(df1['city'].unique())

"""
    AFTER ALL CLEANSED CHECK FOR NULL VALUES AND REPLACE FOR "barcelona"
"""
null_count = df1['city'].isnull().sum()
df1['city'].fillna('barcelona', inplace=True)
# Check if the replacement was successful
null_cities = df1['city'].isnull().sum()
print(f"Number of null values in 'city' column after replacement: {null_cities}")
print(df1['city'].unique())

df2 = df1.copy()

Number of null values in 'city' column after replacement: 0
['barcelona']


/var/folders/yl/d5x0r6t10l52bms12sy70rzc0000gn/T/ipykernel_69836/225649308.py:32: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df1['city'].fillna('barcelona', inplace=True)


# Cleaning date format: 'host_since' and 'first_review'

# Column'first_review' date formated, deal with nulls and replace them for 'listed_date' if valid.

In [9]:
"""
    REFORMAT DATE FOR BOTH first_review AND host_since
"""
df2['first_review'] = pd.to_datetime(df2['first_review'], errors='coerce')
df2['listed_date'] = pd.to_datetime(df2['listed_date'], errors='coerce')
# Check for null values in the 'date' column
print(df2['listed_date'].unique())

"""
    CHECK FOR NULLS IN 'first_review' : IF NULL GIVE THE VALUE OF THE COLUMN host_since
"""
null_first_review = df2['first_review'].isnull().sum()
print(null_first_review )

def fill_first_review_with_host_since(row):
    if pd.isnull(row['first_review']) and pd.notnull(row['listed_date']):
        return row['listed_date']
    return row['first_review']

df2['first_review'] = df2.apply(fill_first_review_with_host_since, axis=1)

# Check if there are still null values in the 'first_review' column
null_count_after = df2['first_review'].isnull().sum()
print(f"Number of null values in 'first_review' after replacement: {null_count_after}")

# There are only 14 so delete those nulls.
null_count = df2['listed_date'].isnull().sum()
print(f"Number of null values in 'host_since'' after replacement: {null_count}")
df2.dropna(subset=['listed_date'], inplace=True)

df3=df2.copy()

/var/folders/yl/d5x0r6t10l52bms12sy70rzc0000gn/T/ipykernel_69836/3924503090.py:5: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df2['listed_date'] = pd.to_datetime(df2['listed_date'], errors='coerce')


<DatetimeArray>
['2010-01-19 00:00:00', '2010-02-18 00:00:00', '2010-04-14 00:00:00',
 '2010-05-29 00:00:00', '2010-06-02 00:00:00', '2010-06-09 00:00:00',
                 'NaT', '2010-01-24 00:00:00', '2010-06-25 00:00:00',
 '2010-07-05 00:00:00',
 ...
 '2019-07-04 00:00:00', '2017-12-28 00:00:00', '2013-11-20 00:00:00',
 '2019-07-05 00:00:00', '2019-07-06 00:00:00', '2019-07-07 00:00:00',
 '2019-07-08 00:00:00', '2015-03-27 00:00:00', '2014-12-26 00:00:00',
 '2019-07-09 00:00:00']
Length: 2803, dtype: datetime64[ns]
13342
Number of null values in 'first_review' after replacement: 19
Number of null values in 'host_since'' after replacement: 33


# Column'host_since' date formated, deal with nulls and replace them for 'first_review' date.

In [10]:
"""
    APPLY DATE TIME FORMATTING
"""
df3['listed_date'] = pd.to_datetime(bcn_df['listed_date'], errors='coerce')
# Check for null values in the 'date' column
null_dates = df3['listed_date'].isnull().sum()
print(null_dates)
print(df1['listed_date'].unique())

"""
    CHECK FOR NULLS IN 'host_since' : IF NULL GIVE THE VALUE OF THE COLUMN 'first_review'
"""
def fill_host_since_with_first(row):
    if pd.isnull(row['listed_date']):
        return row['first_review']
    return row['listed_date']

df3['listed_date'] = df3.apply(fill_host_since_with_first, axis=1)
# Check if there are still null values in the 'first_review' column
null_host = df3['listed_date'].isnull().sum()
print(f"Number of null values in 'first_review' after replacement: {null_host}")

"""
    CRETE 2 NEW COLUMNS FOR 'listed_year' AND 'listed_month'
"""
df3['listed_year'] = df3['listed_date'].dt.year
df3['listed_month'] = df3['listed_date'].dt.month
print(df3[['listed_month', 'listed_year']].head())

df4 = df3.copy()

0
['19/01/2010' '18/02/2010' '14/04/2010' ... '27/03/2015' '26/12/2014'
 '09/07/2019']


/var/folders/yl/d5x0r6t10l52bms12sy70rzc0000gn/T/ipykernel_69836/1956955323.py:4: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df3['listed_date'] = pd.to_datetime(bcn_df['listed_date'], errors='coerce')


Number of null values in 'first_review' after replacement: 0
   listed_month  listed_year
0             1         2010
1             1         2010
2             2         2010
4             4         2010
5             5         2010


# Listing analysis

In [11]:
# Get the top 10 neighborhoods with the highest number of listings
top_10_host = df8['host_id'].value_counts().head(10).index.tolist()

# Assuming df8 is your DataFrame and 'host_since_year' is the column you're interested in
frequency = df4['listed_year'].value_counts().sort_index()  # Sort by year for a chronological plot
x = frequency.index.to_list()
y = list(frequency.values)

fig, ax = plt.subplots()
ax.set_ylabel('id')
ax.set_xlabel('listed_year')
ax.set_title('Number of Hosts per Year')
# Change from bar to plot for a line graph
ax.plot(x, y, marker='o')  # Using markers to highlight data points

plt.show()


NameError: name 'df8' is not defined

In [ ]:
# Assuming df8 is your DataFrame and 'host_since_year' is the column you're interested in
frequency = df4['listed_month'].value_counts().sort_index()  # Sort by year for a chronological plot
x = frequency.index.to_list()
y = list(frequency.values)

fig, ax = plt.subplots()
ax.set_ylabel('id')
ax.set_xlabel('listed_month')
ax.set_title('Number of Hosts per month')
# Change from bar to plot for a line graph
ax.plot(x, y, marker='o')  # Using markers to highlight data points

plt.show()

# Price and cleaning_price : convert to floats and clean format.

In [ ]:
#price and cleaning fee need to be numbers, and null for price deleted
#print(bcn_df['cleaning_fee'].unique())

"""
    REMOVE THE $ SIGN AND OTHER SIMBOLS FOR BOTH PRICE AND CLEANING_FEE:
"""
def convert_to_float(value):
    if isinstance(value, int) or isinstance(value, float):
        return float(value)
    elif isinstance(value, str):
        value = value.replace('$', '').replace(',', '')
        return float(value)
    return value  # In case the value is already NaN or some other type

df4['price'] = df4['price'].apply(convert_to_float)
df4['cleaning_fee'] = df4['cleaning_fee'].apply(convert_to_float).round(2)
print(df4[['price', 'cleaning_fee']].head())
#print(df4['price'].unique())

"""
    CHECK FOR NULLS 
"""
null_price_count = df4['price'].isnull().sum()
print(f"Number of null values in 'price': {null_price_count}")
null_clean_count = df4['cleaning_fee'].isnull().sum()
print(f"Number of null values in cleaning_fee': {null_clean_count}")

"""
    FOR CLEANING_FEE FILL THE NULS. WITH THE AVRG. PER NEIGHBOURHOOD.
"""
neighbourhood_avg_cleaning_fee = df4.groupby('neighbourhood')['cleaning_fee'].transform('mean')

# Fill the null values in the 'cleaning_fee' column with the calculated neighborhood averages
df4['cleaning_fee'].fillna(neighbourhood_avg_cleaning_fee, inplace=True)
overall_avg_cleaning_fee = df4['cleaning_fee'].mean()
# Fill any remaining null values with the overall average cleaning fee
df4['cleaning_fee'].fillna(overall_avg_cleaning_fee, inplace=True)

# Verify that there are no null values left
final_clean_count = df4['cleaning_fee'].isnull().sum()
print(f"Number of null values in 'cleaning_fee' after filling: {final_clean_count}")

df5=df4.copy()

# Check for nul values and duplicates for : 'neighbourhood_cleansed' and 'neighbourhood_grouped_cleansed'

In [ ]:
null = df5['area'].isnull().sum()
print(f"Number of null values in 'area': {null}")

null = df5['neighbourhood'].isnull().sum()
print(f"Number of null values in 'neighbourhood': {null}")
print(bcn_df['area'].nunique())
print(bcn_df['neighbourhood'].nunique())

# Zip_code : clean format and fill nul values 

In [ ]:
#zipcode should be numerical ( fill up nuls with nighbourhood code
#print(bcn_df['zipcode'].unique())
# Clean the 'zipcode' column
"""
    CLEAN 'zip_code' - VALUES SHOULD BE ONLY NUMERICAL
"""

df5['zipcode'] = df5['zipcode'].astype(str)
df5['zipcode'] = df5['zipcode'].str.extract('(\d+)', expand=False)
zipcode_dict = {
    '0': '8000',
    '2015': '8015',
    '13': '8013',
    '28013': '8013',
    '2024': '8024',
    '80001': '8001',
    '80020': '8002',
    '92134': '',
    '9': '8009',
    '806': '8006',
    '2335': '8035',
    '4008009': '8009'
}

# Function to clean and update zipcodes based on the dictionary
def clean_zipcode(zipcode):
    if pd.isnull(zipcode):  # If the value is NaN, return NaN
        return np.nan
    str_zipcode = str(int(zipcode))  # Convert to string without decimal part
    return zipcode_dict.get(str_zipcode, str_zipcode)  # Return the mapped value or the original if not in dictionary

# Apply the clean_zipcode function
df5['zipcode'] = df5['zipcode'].apply(clean_zipcode)
df5['zipcode'] = pd.to_numeric(df5['zipcode'], errors='coerce')
df5['zipcode'] = df5['zipcode'].astype('Int64')
null_zipcode_count = df5['zipcode'].isnull().sum()

# Create a mapping of neighbourhood_cleansed to get zipcode
zipcode_mapping = df5.groupby('neighbourhood')['zipcode'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan).to_dict()


"""
    FUNCTION TO FILL MISSING ZIPCODES  REFERING TO THE NEIGHBOURHOODS.
"""
def fill_missing_zipcode(row, mapping):
    if pd.isnull(row['zipcode']):
        return mapping.get(row['neighbourhood'], np.nan)
    return row['zipcode']

df5['zipcode'] = df5.apply(fill_missing_zipcode, axis=1, mapping=zipcode_mapping)

"""
    CHECK FOR NUL VALUES AFTER FILLING
"""
# Check for null values in 'zipcode' after filling
null_zipcode_count_after = df5['zipcode'].isnull().sum()
print(f"Number of null values in 'zipcode' after filling nulls: {null_zipcode_count_after}")

df7=df5.copy()

# Clean the amenities column

In [ ]:

# Split the amenities column into lists
df7['amenities'] = df7['amenities'].str.split(',')

# Add a new column with the count of amenities
df7['amenities_count'] = df7['amenities'].apply(len)

# Print the updated DataFrame
print("Updated DataFrame with amenities count:")
#print(df7)

# Missing values for bedrooms, beds, and bathrooms are replaced with the median.

In [ ]:
"""
    FILL MISSING INFO FOR NUMBER OD BEDROOMS, BEDS AND BATHROOM USING THE MEDIAN
"""

df7.room_type.value_counts()
df7.drop('room_type', axis=1, inplace=True)
for col in ['bathrooms', 'bedrooms', 'beds']:
    df7[col].fillna(df7[col].median(), inplace=True)


# Handle null values by filling with the mean value of the corresponding 'neighbourhood_cleansed'
bcn_df['bathrooms'] = bcn_df.groupby('neighbourhood_cleansed')['bathrooms'].transform(lambda x: x.fillna(x.mean()))
bcn_df['bedrooms'] = bcn_df.groupby('neighbourhood_cleansed')['bedrooms'].transform(lambda x: x.fillna(x.mean()))
bcn_df['beds'] = bcn_df.groupby('neighbourhood_cleansed')['beds'].transform(lambda x: x.fillna(x.mean()))

# Convert columns to integer data type
bcn_df['bathrooms'] = bcn_df['bathrooms'].astype(int)
bcn_df['bedrooms'] = bcn_df['bedrooms'].astype(int)
bcn_df['beds'] = bcn_df['beds'].astype(int)

# Check the data types after conversion
#print(bcn_df.dtypes)
# Check for null values in 'zipcode' after filling
null_zipcode_count_after = bcn_df['bathrooms'].isnull().sum()
null_zipcode_count_after2 = bcn_df['bedrooms'].isnull().sum()
null_zipcode_count_after3 = bcn_df['beds'].isnull().sum()
print(f"Number of null values in 'zipcode' after filling nulls: {null_zipcode_count_after}")
print(f"Number of null values in 'zipcode' after filling nulls: {null_zipcode_count_after2}")
print(f"Number of null values in 'zipcode' after filling nulls: {null_zipcode_count_after3}")

# Create new column: tourist_atractions

In [ ]:
"""
    DEFINE A DICTONARY WITH ALL THE MAIN TOURIS ATTRACTIONS BY NEIGHBOURHOOD, CREATE A NEW COLUMN TO BE ADDED ON THE DF, AND ANOTHER FOR THE TOTAL OF ATRACTIONS.
"""
tourist_attractions = {
    'Sant Martí': ['Playa Poblenou', 'Museo de Ciencias Naturales','Torre Glòries', 'Museo del Diseño'],
    'Eixample': ['Casa Batlló', 'Casa Milà', 'Passeig de Gràcia', 'Mercado de Sant Antoni', 'Mercado de los Encants','El Auditorio','Teatro Nacional de Cataluña','Sagrada Familia', 'Las Arenas', 'La plaza de Catalunya','La Casa de les Punxes'],
    'Gràcia': ['Park Güell', 'Vila de Gràcia', 'El paseo de Sant Joan', 'La Casa Vicens','La Casa Fuster'],
    'Horta-Guinardó': ['Parc del Laberint dHorta','Hospital de Sant Pau','Collserola'],
    'Les Corts': ['Camp Nou', 'Jardines de Pedralbes', 'Los Pabellones de la Finca Güell', 'El Real Monasterio de Santa Maria de Pedralbes'],
    'Ciutat Vella': ['La Rambla', 'Barrio Gótico', 'Museo Picasso', 'Catedral de Barcelona', 'Mercado Sant Pere y Santa Caterina', 'Palau de la Musica','Santa Maria del Mar','El Born y la Ribera','La Barceloneta', 'El Arco de Triunfo', 'Drassanes', 'El Mercado de la Boqueria', 'La plaza del Rei', 'La plaza Reial', 'El Palau de Mar','El Gran Teatre del Liceu', 'El Palau Güell', 'El parque de la Ciutadella', 'La playa de la Barceloneta', 'El Zoo de Barcelona'],
    'Sants-Montjuïc': ['Montjuïc', 'Fuente Magica', 'Poble Espanyol', 'El Museo Nacional de Arte de Catalunya', 'Sants y Hostafrancs', 'La plaza de Espanya', 'El Jardín Botánico - Museu de Ciències Naturals de Barcelona', 'CaixaForum', 'El Pabellón Mies van der Rohe - Institut Municipal Fundació Mies van der Rohe', 'El teleférico de Montjuïc', 'El Teatre Grec', 'El Palacio de Sant Jordi', 'El Estadio Olímpico'],
    'Sarrià-Sant Gervasi': ['CosmoCaixa', 'Parque Tibidabo'],
    'Nou Barris': ['El parque Central de Nou Barris'],
    'Sant Andreu': ['Fabra i Coats']
}

def get_tourist_attractions(neighbourhood):
    return tourist_attractions.get(neighbourhood, [])

df7['tourist_attractions'] = df7['neighbourhood_group_cleansed'].apply(lambda x: get_tourist_attractions(x))
df7['total_tourist_attractions'] = df7['tourist_attractions'].apply(len)

#print(df7.head())

df8= df7.copy()

In [ ]:

# Calculate the average price per neighbourhood_group_cleansed
avg_price = df8.groupby('area')['total_tourist_attractions'].count().sort_values()

# Get the neighbourhood groups and their average prices
x = avg_price.index.to_list()
y = avg_price.values

# Create a horizontal bar chart
fig, ax = plt.subplots(figsize=(6, 4))
ax.set_xlabel('Average Price x night', fontsize=1)
ax.set_ylabel('Area ',fontsize=2)

# Plot horizontal bars with grey color
ax.barh(x, y)
ax.tick_params(axis='both', which='major', labelsize=10)
# Display the plot
plt.show()

# Analysis of listings: year of creationg, listings per neighbourhood, top owners

In [ ]:
df8['host_id'].nunique()

# Get the top 10 neighborhoods with the highest number of listings
top_10_host = df8['host_id'].value_counts().head(10).index.tolist()

frequency = df8['listed_year'].value_counts()
x = frequency.index.to_list()
y = list(frequency.values)
fig, ax = plt.subplots()
ax.set_ylabel('id')
ax.set_xlabel('listed_year')
plt.bar(x,y)
plt.show()

# Compute the frequency of 'host_since_year'
frequency = bcn_df['listed_year'].value_counts().sort_index()

# Convert the result to lists for plotting
x = frequency.index.to_list()
y = list(frequency.values)

# Plot the frequency as a line plot
fig, ax = plt.subplots(figsize=(6, 2))
ax.plot(x, y, marker='o', linestyle='-', color='b')
ax.set_ylabel('Number of Hosts')
ax.set_xlabel('Host Since Year')
ax.set_title('Number of Hosts per Year')
plt.xticks(rotation=45)
plt.grid(True)

# Show the plot
plt.show()

# Analysis : price per neighbourhood grouped

In [ ]:
# Calculate descriptive statistics for 'price' by 'neighbourhood'
stats = df8.groupby('area')['price'].describe().round()
# Pivot table with descriptive statistics
pivot_price_neighbourhood = df8.pivot_table(values='price', index='area', aggfunc=['mean','min', 'max'])

# Rename the columns to match the output of describe()
pivot_price_neighbourhood.columns = [ 'mean', 'min', 'max']

# Round the values to the nearest integer
pivot_price_neighbourhood = pivot_price_neighbourhood.round()

# Print the pivot table
print("\nPivot table with descriptive statistics:")
print(pivot_price_neighbourhood)

In [ ]:
# NEIGHBOURHOODS WITH HIGHEWR AVERAGE PRICE

# Calculate the average price per neighbourhood_group_cleansed
avg_price = df8.groupby('area')['price'].mean().head(10).sort_values()

# Get the neighbourhood groups and their average prices
x = avg_price.index.to_list()
y = avg_price.values

# Create a horizontal bar chart
fig, ax = plt.subplots(figsize=(6, 4))
ax.set_xlabel('Average Price x night', fontsize=1)
ax.set_ylabel('Area ',fontsize=2)

# Plot horizontal bars with grey color
ax.barh(x, y)
ax.tick_params(axis='both', which='major', labelsize=10)
# Display the plot
plt.show()
#avg_price = df8.groupby('neighbourhood_group_cleansed')['price'].mean().head(10).sort_values()

In [ ]:
# NEIGHBOURHOODS WITH HIGHEWR AVERAGE PRICE

# Calculate the average price per neighbourhood_group_cleansed
avg_price = df8.groupby('area')['price'].median().head(10).sort_values()

# Get the neighbourhood groups and their average prices
x = avg_price.index.to_list()
y = avg_price.values
#plt.figure(figsize=(4, 2))
# Create a horizontal bar chart
fig, ax = plt.subplots(figsize=(6, 4))
ax.set_xlabel('Average Price x night', fontsize=1)
ax.set_ylabel('Neighbourhood Group ', fontsize=2)

# Plot horizontal bars with grey color
ax.barh(x, y)
ax.tick_params(axis='both', which='major', labelsize=10)
# Display the plot
plt.show()
#avg_price = df8.groupby('neighbourhood_group_cleansed')['price'].mean().head(10).sort_values()

In [ ]:
# Calculate descriptive statistics for 'listings' by numero de guests

stats = df8.groupby('accommodates')['price'].describe().round()
# Pivot table with descriptive statistics
pivot_price_accommodates = df8.pivot_table(values='price', index='accommodates', aggfunc=['mean', 'min', 'max'])

# Rename the columns to match the output of describe()
pivot_price_accommodates.columns = ['mean','min', 'max']

# Round the values to the nearest integer
pivot_price_accommodates = pivot_price_accommodates.round()

# Print the pivot table
print("\nPivot table with descriptive statistics (no decimals):")
print(pivot_price_accommodates)

In [ ]:

# Create a DataFrame of listing counts and median price by neighbourhood_group_cleansed
neighbourhood_df = df8.groupby('area').agg(
    number_of_listings=('id', 'size'),
    average_price=('price', 'median')
).reset_index()

# Set the neighbourhood_group_cleansed column as the index
neighbourhood_df.set_index('neighbourhood_group_cleansed', inplace=True)

# Sort the DataFrame by average_price
neighbourhood_df_sorted = neighbourhood_df.sort_values('average_price', ascending=False)

# Plot number of listings and median price in one bar plot
plt.rcParams['font.size'] = 15
plt.rcParams['figure.figsize'] = 15,5

fig, ax1 = plt.subplots()

# Plot number of listings as bar plot
neighbourhood_df_sorted['number_of_listings'].plot(kind='bar', ax=ax1, position=0, width=0.4, color='darkgrey', label='Number of Listings')

# Create a second y-axis for the average price
ax2 = ax1.twinx()
neighbourhood_df_sorted['average_price'].plot(kind='bar', ax=ax2, position=1, width=0.4, color='lightgrey', label='Average Price')

# Set the labels and title
ax1.set_xlabel('Neighbourhood Group')
ax1.set_ylabel('Number of Listings', color='black')
ax2.set_ylabel('Average Price', color='black')
ax1.set_xticklabels(neighbourhood_df_sorted.index, rotation=90)
plt.title('Number of Listings and Average Price by Neighbourhood Group')

lines_1, labels_1 = ax1.get_legend_handles_labels()
lines_2, labels_2 = ax2.get_legend_handles_labels()
ax1.legend(lines_1 + lines_2, labels_1 + labels_2, loc='lower left', bbox_to_anchor=(1.05, 1))

plt.show()


In [ ]:

# Creating a dataframe of listing counts and median price by borough
neighbourhood_df = df8.groupby('area').size()
neighbourhood_df.rename(columns={0: 'number_of_listings'}, inplace=True)
neighbourhood_df['average_price'] = df.groupby('area').price.median().values

# Plot number of listings and median price in one bar plot
plt.rcParams['font.size'] = 15
plt.rcParams['figure.figsize'] = 25, 5
neighbourhood_df_sorted = neighbourhood_df.sort_values('average_price', ascending=False)
neighbourhood_df_sorted.plot( kind= 'bar' , secondary_y= 'average_price' , rot= 90 )
plt.xticks(rotation='vertical')

plt.show()

# Analysys regarding the type of apartment

In [12]:
# Calculate descriptive statistics for 'listings' by room type
stats = df8.groupby('room_type')['price'].describe().round()
# Pivot table with descriptive statistics
pivot_price_accommodates = df8.pivot_table(values='price', index='room_type', aggfunc=['mean', 'min', 'max'])

# Rename the columns to match the output of describe()
pivot_price_accommodates.columns = ['mean','min', 'max']

# Round the values to the nearest integer
pivot_price_accommodates = pivot_price_accommodates.round()

# Print the pivot table
print("\nPivot table with descriptive statistics (no decimals):")
print(pivot_price_accommodates)

NameError: name 'df8' is not defined

In [ ]:
#what type of properties
props_by_type = df8.dropna(subset=["room_type"]).groupby("room_type")["id"].agg(["count"])
plt.figure(figsize=(14,5))
x = props_by_type.index
y = props_by_type["count"]
plt.bar(x, y)
plt.xticks(x, x, rotation="vertical")
plt.ylabel("Number of properties")
plt.xlabel("Type of property")
plt.title("Distribution of properties based in its type")
plt.show()

# How are different numeric attributes of property correlated to its booking price?

In [ ]:
atractions = df8.dropna(subset=["amenities_count", "price"])

x = atractions["amenities_count"]
y = atractions["price"]

plt.figure(figsize=(4, 2))  # Adjust the figsize as needed
plt.scatter(x, y)

plt.xlabel("Number of amenities in the apartment")
plt.ylabel("Booking price per night")
plt.title("Booking price against number of amenities in the apartment")

plt.show()

In [ ]:


atractions = df8.dropna(subset=["total_tourist_attractions", "price"])

x = atractions["total_tourist_attractions"]
y = atractions["price"]

plt.figure(figsize=(4, 2))  # Adjust the figsize as needed
plt.scatter(x, y)

plt.xlabel("Number of tourist attractions nearby")
plt.ylabel("Booking price per night")
plt.title("Booking price against number of of tourist attractions nearby")

plt.show()

In [ ]:
# NUMBER OF BEDROOMS IN PROPERTY

bathroom_df = df8[df8['room_type'] == 'Entire home/apt']

bed = bathroom_df.dropna(subset=["bathrooms", "price"])

# Extract the 'bedrooms' and 'price' columns
x = bed["bathrooms"]
y = bed["price"]

# Create a smaller scatter plot
plt.figure(figsize=(4, 2))  # Adjust the figsize as needed
plt.scatter(x, y)

# Set labels and title
plt.xlabel("Number of bathrooms in property")
plt.ylabel("Booking price per night")
plt.title("Booking price against number of bathrooms in property")

# Show the plot
plt.show()

In [ ]:

room_df = df8[df8['room_type'] == 'Entire home/apt']

# Drop rows with missing values in 'bedrooms' and 'price'
bed = room_df.dropna(subset=['bedrooms', 'price'])

# Extract the relevant columns
x = bed['bedrooms']
y = bed['price']

# Plotting
plt.figure(figsize=(4, 2))  # Adjust the figsize as needed
plt.scatter(x, y)

plt.xlabel("Number of bedrooms in property")
plt.ylabel("Booking price per night")
plt.title("Booking price against number of bedrooms in property")

plt.show()

In [ ]:

bathroom_df = df8[df8['room_type'] == 'Entire home/apt']
bath = df8.dropna(subset=["bathrooms", "price"])

# Extract the 'bedrooms' and 'price' columns
x = bath["bathrooms"]
y = bath["price"]

# Create a smaller scatter plot
plt.figure(figsize=(4, 2))  # Adjust the figsize as needed
plt.scatter(x, y)

# Set labels and title
plt.xlabel("Number of bathrooms in property")
plt.ylabel("Booking price per night")
plt.title("Booking price against number of bathrooms in property")

# Show the plot
plt.show()

# reviews

In [ ]:
# Checking the distributions of the review ratings columns
variables_to_plot = list(df8.columns[df8.columns.str.startswith("review_scores") == True])
fig = plt.figure(figsize=(8,6))
for i, var_name in enumerate(variables_to_plot):
    ax = fig.add_subplot(3,3,i+1)
    df8[var_name].hist(bins=10,ax=ax)
    ax.set_title(var_name)
fig.tight_layout()
plt.show()

In [ ]:
# do code to calculate the  following statistics for some numerical columns:
columns_to_calculate = ['price']
# Calculate overall statistics
overall_stats = df8[columns_to_calculate].describe()

# Calculate statistics by 'neighbourhood_cleansed'
stats_by_neighbourhood_cleansed = df8.groupby('neighbourhood_cleansed')[columns_to_calculate].describe()

# Calculate statistics by 'neighbourhood_group_cleansed'
stats_by_neighbourhood_group_cleansed = df8.groupby('neighbourhood_group_cleansed')[columns_to_calculate].describe()

# Display overall statistics
print("Overall Statistics:")
print(overall_stats)

# Display statistics by 'neighbourhood_cleansed'
print("\nStatistics by 'neighbourhood_cleansed':")
print(stats_by_neighbourhood_cleansed)

# Display statistics by 'neighbourhood_group_cleansed'
print("\nStatistics by 'neighbourhood_group_cleansed':")
print(stats_by_neighbourhood_group_cleansed)


# Calculate statistics for 'price_per_night'
price_per_night_stats = df8['price_per_night'].describe()

# Calculate statistics for 'price_per_night' by 'neighbourhood_cleansed'
price_per_night_by_neighbourhood_cleansed = df8.groupby('neighbourhood_cleansed')['price_per_night'].describe()

# Calculate statistics for 'price_per_night' by 'neighbourhood_group_cleansed'
price_per_night_by_neighbourhood_group_cleansed = df8.groupby('neighbourhood_group_cleansed')['price_per_night'].describe()


In [ ]:
import pandas as pd
import seaborn as sb
import plotly.express as px

# Datos con latitud/longitud y valores. Los nombres de columna son latitude, longitude y tot_pob


fig = px.density_mapbox(df8, lat = 'latitude', lon = 'longitude', z = 'id',
                        radius = 2,
                        center = dict(lat = 42.83, lon = -8.35),
                        zoom = 6,
                        mapbox_style = 'open-street-map')
fig.show()

In [ ]:
import plotly.express as px
import plotly.io as pio
#import kaleidoimport plotly.express as px

# Load your dataset
# Replace 'path_to_your_dataset.csv' with the actual path to your dataset


# Ensure the dataset has the required columns
# The dataset should have columns like 'latitude', 'longitude', 'name', 'price', etc.

# Create a Plotly scatter mapbox
fig = px.scatter_mapbox(df8, 
                        lat="latitude", 
                        lon="longitude", 
                        hover_name="id",  # The column you want to show on hover (e.g., listing name)
                        hover_data={"price": True, "neighbourhood_group_cleansed": True},  # Additional data to show on hover
                        color_discrete_sequence=["fuchsia"], 
                        zoom=6, 
                        height=300)

# Update the layout of the map
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

# Show the map
fig.show()

In [ ]:
"""
 NEIGHBOURHOODS  WITH MORE LISTINGS
"""
# Get the value counts for 'neighbourhood_group_cleansed'
# Get the value counts for 'neighbourhood_group_cleansed'

value_count = df8['neighbourhood'].value_counts().head(10).sort_values(ascending=True)
x = value_count.index.to_list()
y = value_count.values

# Create a horizontal bar chart
fig, ax = plt.subplots()
ax.set_xlabel('Number of listings')
ax.set_ylabel('Neighbourhoods')

# Plot horizontal bars
ax.barh(x, y)

# Display the plot
plt.show()

In [ ]:
"""
 NEIGHBOURHOODS  WITH HIGHER AVERAGE PRICE
"""

# Calculate the average price per neighbourhood_group_cleansed
avg_price = df8.groupby('neighbourhood')['price'].mean().head(10).sort_values()

# Get the neighbourhood groups and their average prices
x = avg_price.index.to_list()
y = avg_price.values

# Create a horizontal bar chart
fig, ax = plt.subplots()
ax.set_xlabel('Average Price x night')
ax.set_ylabel('Neighbourhood Group ')

# Plot horizontal bars with grey color
ax.barh(x, y)

# Display the plot
plt.show()

In [ ]:
"""
 NEIGHBOURHOODS WITH HIGHER SCORE REVIEWS
"""

# Get the top 10 neighborhoods with the highest number of listings
top_10_neighborhoods = df8['neighbourhood'].value_counts().head(10).index.tolist()

# Filter the DataFrame to include only the top 10 neighborhoods
df_top_10 = df8[df8['neighbourhood'].isin(top_10_neighborhoods)]

# Calculate the average price per night for each neighborhood
average_price_per_night = df_top_10.groupby('neighbourhood')['price'].mean().sort_values()

# Plot the average price per night for each neighborhood
average_price_per_night.plot(kind='bar', figsize=(10, 6))
plt.xlabel('Neighborhood')
plt.ylabel('Average Price per Night')
plt.title('Average Price per Night for Top 10 Neighborhoods')
plt.xticks(rotation=45)
plt.show()


In [ ]:
df8['host_id'].nunique()

In [ ]:
#how does the price vary regarding the number of a tourist place to visit

sil = df8.dropna(subset=['total_tourist_attractions', 'price'])

# Convert 'total_tourist_attractions' to integer
sil['total_tourist_attractions'] = sil['total_tourist_attractions'].astype(int)

# Extract the 'total_tourist_attractions' and 'price' columns
x = sil['total_tourist_attractions']
y = sil['price']

# Create the scatter plot
plt.scatter(x, y)

# Set labels and title
plt.xlabel("Number of tourist attractions nearby")
plt.ylabel("Booking price")
plt.title("Booking price vs number of tourist attractions nearby")

# Show the plot
plt.show()

# 9. How are different numeric attributes of property correlated to its booking price?

In [ ]:
bed = df8.dropna(subset=["bedrooms", "price"])

# Extract the 'bedrooms' and 'price' columns
x = bed["bedrooms"]
y = bed["price"]

# Create a smaller scatter plot
plt.figure(figsize=(4, 2))  # Adjust the figsize as needed
plt.scatter(x, y)

# Set labels and title
plt.xlabel("Number of bedrooms in property")
plt.ylabel("Booking price per night")
plt.title("Booking price against number of bedrooms in property")

# Show the plot
plt.show()

In [ ]:
bed = df8.dropna(subset=["bathrooms", "price"])

# Extract the 'bedrooms' and 'price' columns
x = bed["bathrooms"]
y = bed["price"]

# Create a smaller scatter plot
plt.figure(figsize=(4, 2))  # Adjust the figsize as needed
plt.scatter(x, y)

# Set labels and title
plt.xlabel("Number of bathrooms in property")
plt.ylabel("Booking price per night")
plt.title("Booking price against number of bathrooms in property")

# Show the plot
plt.show()

# Group by 'neighbourhood' and calculate the mean of the specified columns
grouped = df8.groupby('neighbourhood_group_cleansed').mean()

# Extract data for plotting
neighbourhoods = grouped.index.values
cleaning_scores = grouped['review_scores_cleanliness'].values
location_scores = grouped['review_scores_location'].values
value_scores = grouped['review_scores_value'].values
total_scores = grouped['total score'].values

# Plotting
X_axis = np.arange(len(neighbourhoods))

plt.figure(figsize=(10, 6))
plt.bar(X_axis - 0.3, cleaning_scores, 0.2, label='Cleaning Score')
plt.bar(X_axis - 0.1, location_scores, 0.2, label='Location Score')
plt.bar(X_axis + 0.1, value_scores, 0.2, label='Value Score')
plt.bar(X_axis + 0.3, total_scores, 0.2, label='Total Score')

plt.xticks(X_axis, neighbourhoods, rotation=45)
plt.xlabel("Neighbourhood")
plt.ylabel("Average Scores")
plt.title("Average Scores by Neighbourhood")
plt.legend()
plt.show()

In [ ]:
# Visualizing the distribution for every "feature"
df8.hist(edgecolor="black", linewidth=1.2, figsize=(30, 30));

In [ ]:
# top host cuantos listing

# Get the top 10 neighborhoods with the highest number of listings
top_10_host = df8['host_id'].value_counts().head(10).index.tolist()

# Filter the DataFrame to include only the top 10 neighborhoods
df_top_10 = df8[df8['host_id'].isin(top_10_host)]

# Calculate the average price per night for each neighborhood
average_price_per_night = df_top_10.groupby('neighbourhood_cleansed')['price'].mean().sort_values()

# Plot the average price per night for each neighborhood
average_price_per_night.plot(kind='bar', figsize=(10, 6))
plt.xlabel('Neighborhood')
plt.ylabel('Average Price per Night')
plt.title('Average Price per Night for Top 10 Neighborhoods')
plt.xticks(rotation=45)
plt.show()



fig, ax = plt.subplots()
#y = grouped_location['VOLUSIA COUNTY']

selected_neigh = ['Eixample', 'Sant Martí', 'Sants-Montjuïc', 'Les Corts']

# Filter the DataFrame for selected counties
df_filtered = df8[df8['neighbourhood_grouped_cleansed'].isin(selected_neigh)]

# Pivot the data to get incidents per season for each county
pivot_df = df_filtered.pivot(index='neighbourhood_grouped_cleansed', columns='property_type', values='id')

# Fill NaN with 0 if necessary
pivot_df = pivot_df.fillna(0)

# Extract the data for plotting
counties = pivot_df.index.tolist()
fall = pivot_df['fall'].tolist()
spring = pivot_df['spring'].tolist()
summer = pivot_df['summer'].tolist()
winter = pivot_df.get('winter', [0]*len(counties)).tolist()  # If winter is not present, fill with zeros

# Number of counties
X_axis = np.arange(len(counties))

# Plotting
fig, ax = plt.subplots()

# Width of bars
bar_width = 0.2

# Creating bars
plt.bar(X_axis - bar_width * 1.5, fall, bar_width, label='fall')
plt.bar(X_axis - bar_width * 0.5, spring, bar_width, label='spring')
plt.bar(X_axis + bar_width * 0.5, summer, bar_width, label='summer')
plt.bar(X_axis + bar_width * 1.5, winter, bar_width, label='winter')

# Customizing the plot
plt.xticks(X_axis, counties)
plt.xlabel("County")
plt.ylabel("Total number of incidents")
plt.legend()
plt.title("Incidents per Season by County")
plt.show()

In [ ]:
pivot_room_type =df8.pivot_table(index="room_type", columns="neighbourhood_group_cleansed", values="price", aggfunc='mean')
styled_pivot_table = (pivot_room_type.style.format("{:.0f}"))
styled_pivot_table

In [ ]:
lil = df8['price'].isnull().sum()
print(f"Number of null values in 'zipcode' after filling nulls: {lil}")

pivot_list_df = df8.pivot_table(index='neighbourhood_group_cleansed', columns='host_since_year', values='listing_url', aggfunc='count')

# Apply the function and format the values as integers
styled_pivot_list = (pivot_list_df.style.format("{:.0f}"))  # This formats the numbers as integers

# Display the styled pivot table
styled_pivot_list


# Create the pivot table


In [ ]:
frequency = df8['host_since_year'].value_counts()
x = frequency.index.to_list()
y = list(frequency.values)
fig, ax = plt.subplots()
ax.set_ylabel('id')
ax.set_xlabel('host_since_year')
plt.bar(x,y)
plt.show()

In [ ]:
neighbourhood_groups = bcn_df['neighbourhood_group_cleansed'].unique()

for group in neighbourhood_groups:
    # Filter the DataFrame for the current group
    group_df = bcn_df[bcn_df['neighbourhood_group_cleansed'] == group]
    
    # Calculate the frequency of 'host_since_year' for the current group
    frequency = group_df['host_since_year'].value_counts().sort_index()
    x = frequency.index.to_list()
    y = list(frequency.values)
    
    # Plot the frequency of 'host_since_year' for the current group
    fig, ax = plt.subplots()
    ax.set_ylabel('Count')
    ax.set_xlabel('host_since_year')
    ax.set_title(f'Host Since Year for {group}')
    plt.bar(x, y)
    plt.show()

In [ ]:
#fig, ax = plt.subplots((figsize=(10, 8)))
#y = grouped_location['VOLUSIA COUNTY']

#x = list(grouped_location['Year'].index.values)
#y = grouped_location['Year'].iloc[:, 0].value_counts().index.to_list()
X_axis = np.arange(len(x))

plt.bar(X_axis - 0.4, y1, 0.2, label = 'fall')
plt.bar(X_axis - 0.2, y2, 0.2, label = 'spring')
plt.bar(X_axis + 0.0, y3, 0.2, label = 'summer')
plt.bar(X_axis + 0.2, y4, 0.2, label = 'winter')

#ax.set_xticklabels(x,rotation=45)
plt.xticks(X_axis, x)
plt.xlabel("County")
plt.ylabel("Total number of incidents")

plt.legend()
plt.show()



fig, ax = plt.subplots(figsize=(14, 10))  # Adjust the size as needed

# Create horizontal bar plot
ax.barh(x, y, color='skyblue', edgecolor='black')

# Add labels and title
ax.set_xlabel('Average Price ($)')
ax.set_ylabel('Neighbourhood')
ax.set_title('Average Price per Neighbourhood in Barcelona')

# Customize ticks
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Add value labels to each bar
for i in range(len(y)):
    ax.text(y[i], i, f'${y[i]:.2f}', va='center', ha='left', fontsize=10)

# Show the plot
plt.show()

In [ ]:
#we have the column bes and the column beedroom , i ned to add a new column that calculates the ratio of beds per bedroom, and that info goes in a nwe column called beds_ratio
#we have the column bes and the column beedroom , i ned to add a new column that calculates the ratio of beds per bedroom, and that info goes in a nwe column called beds_ratio

ratio de ba;os por piso y de cuantos caben por piso por barrio


# Calculate the ratio of beds per bedroom and store it in a new column
bcn_df['beds_ratio'] = bcn_df['beds'] / bcn_df['bedrooms']

# Show the updated DataFrame
#print(bcn_df.head())
#print(bcn_df['beds_ratio'].unique())

bcn_df['beds_ratio'] = (bcn_df['beds'] / bcn_df['bedrooms']).round(2)

# Replace 'inf' values with NaN
bcn_df['beds_ratio'] = bcn_df['beds_ratio'].replace(np.inf, np.nan).round(2)

# Fill NaN values in 'beds_ratio' column with the mean value of the corresponding 'neighbourhood_cleansed'
bcn_df['beds_ratio'] = bcn_df.groupby('neighbourhood_cleansed')['beds_ratio'].transform(lambda x: x.fillna(x.mean().round(2)))

# Show unique values in the 'beds_ratio' column
print(bcn_df['beds_ratio'].unique())

#after calculating the ratio it mus be in all cases an intiger (2 decimlas). Then for null values, fill the null value looking at the average value in the column neighbourhood_cleansed, so for each neighbourhood_cleansed with a null give the average for the same neighbourhood_cleansed  